In [1]:
import numpy as np
import tensorflow as tf
import req
import time

In [2]:
class Client():
    def __init__(self, name, X, y, learning_rate):
        self.name = name
        self.X = tf.constant(X)
        self.y = tf.constant(y)
        self.W = tf.Variable(np.zeros((2, 1)))
        self.b = tf.Variable(0, dtype=tf.double)
        self.round = 0
        self.epochs = 100
        self.learning_rate = learning_rate
    
    def check_trian(self):
        curr_r = req.get_round()
        if curr_r > self.round:
            self.round = curr_r
            self.update_local_model()
            self.train()
            self.send_local_model()
            print("Trained global model.")
        else:
            print("Round is stil going on.")
    
    def update_local_model(self):
        global_p = req.get_pre_global()
        W, b = global_p['W'], global_p['b']
        self.W = tf.Variable(np.array(W).reshape((2, 1)))
        self.b = tf.Variable(b, dtype=tf.double)

    def train(self):
        for _ in range(self.epochs):
            with tf.GradientTape() as tape:
                tape.watch([self.W, self.b])
                phi = self.X @ self.W + self.b
                pred = tf.keras.activations.sigmoid(phi)
                loss = tf.keras.losses.binary_crossentropy(self.y, pred, axis=0)

            grad = tape.gradient(loss, {
                "W" : self.W,
                "b" : self.b
            })
            self.W = self.W - self.learning_rate * grad['W']
            self.b = self.b - self.learning_rate * grad['b']
        print(loss.numpy()[0])
    
    def convert_params(self):
        W_temp = self.W.numpy().tolist()
        W = [w[0] for w in W_temp]
        b = self.b.numpy().tolist()
        return W, b
    
    def send_local_model(self):
        param_id = "local_" + str(self.round) + "_" + self.name
        W, b = self.convert_params()
        req.post_param(param_id, W, b)
            
        

In [3]:
data = np.load('./data2d.npz')
X, y = data['X'], data['y']

In [4]:
client_2 = Client("2", X[len(X)//2:], y[len(X)//2:].reshape((35, 1)), 1e-2)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-23 15:54:38.366441: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-23 15:54:38.366761: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
client_2.check_trian()

0.2530861709327289
Trained global model.


In [6]:
for i in range(100):
    time.sleep(5)
    client_2.check_trian()
    

0.17779173452958144
Trained global model.
0.14189486617276076
Trained global model.
0.12149112207146102
Trained global model.
0.10795511109493124
Trained global model.
0.09815960567391627
Trained global model.
0.09065106683220536
Trained global model.
0.08465614034043084
Trained global model.
0.07972238044501621
Trained global model.
0.0755658944935677
Trained global model.
0.07199868319396296
Trained global model.
0.06889079945221205
Trained global model.
0.06614923330208912
Trained global model.
0.06370545586558404
Trained global model.
0.06150772642959458
Trained global model.
0.05951615468703701
Trained global model.
0.05769942567917381
Trained global model.
0.05603256519909634
Trained global model.
0.054495377032360785
Trained global model.
0.053071326074900387
Trained global model.
0.051746724612092086
Trained global model.
0.05051012920324955
Trained global model.
0.049351886715175604
Trained global model.
0.04826378782691107
Trained global model.
0.04723879919733025
Trained glo